In [1]:
from glycan import glycan
import numpy as np
import pandas as pd

from sklearn import svm
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt

In [2]:
g = glycan()
numberedExcel1 = g.getData('../../old_sheet1.csv', wh='old', totalTabs=3)
numberedExcel2 = g.getData('../../old_sheet2.csv', wh='old', totalTabs=4)
numberedExcel3 = g.getData('../../old_sheet3.csv', wh='old', totalTabs=4)
numberedExcel4 = g.getData('../../new_sheet1.csv', totalTabs=2)
numberedExcel5 = g.getData('../../new_sheet2.csv', totalTabs=2)
numberedExcel6 = g.getData('../../new_sheet3.csv', totalTabs=2)
numberedExcel7 = g.getData('../../new_sheet4.csv', totalTabs=4)
numberedExcel8 = g.getData('../../new_sheet5.csv', totalTabs=4)
numberedExcel9 = g.getData('../../new_sheet6.csv', totalTabs=4)

# This file has 2 tabs, but we only read one. The second tab has been updated into a different file
#numberedExcel10 = g.getData('../Results_Feb2021/egg_virus_dilutions.csv', totalTabs=1)
numberedExcel10 = g.getData('./egg_virus_dilutions_secondTab_updated.csv', totalTabs=2)

numberedExcel_temp = pd.concat([numberedExcel1, numberedExcel2, numberedExcel3, numberedExcel4, numberedExcel5, numberedExcel6, numberedExcel7, numberedExcel8, numberedExcel9, numberedExcel10], axis=0)
#numberedExcel_temp = pd.concat([numberedExcel1, numberedExcel2, numberedExcel3, numberedExcel4, numberedExcel7, numberedExcel8, numberedExcel9, numberedExcel10], axis=0)

# Removing the blocks that were hand-picked by Taryn
egg = numberedExcel_temp[~((numberedExcel_temp.SubArr == 23) | (numberedExcel_temp.SubArr == 25) | (numberedExcel_temp.SubArr == 26))]

egg.reset_index(drop=True, inplace=True)
egg['MVF'] = egg.MVF/egg.GlycDen

For now, we are skipping the header
For now, we are skipping the header
For now, we are skipping the header
For now, we are skipping the header
For now, we are skipping the header
For now, we are skipping the header
For now, we are skipping the header
For now, we are skipping the header
For now, we are skipping the header
For now, we are skipping the header


/Users/chitrakgupta/Desktop/ASU_projects/GlycanModification/Data_Mar2020/find_examples/Results_Mar2021/glycan/_read.py:162: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



For now, we are skipping the header
For now, we are skipping the header
For now, we are skipping the header
For now, we are skipping the header
For now, we are skipping the header
For now, we are skipping the header
For now, we are skipping the header
For now, we are skipping the header
For now, we are skipping the header
For now, we are skipping the header
For now, we are skipping the header
For now, we are skipping the header
For now, we are skipping the header
For now, we are skipping the header
For now, we are skipping the header
For now, we are skipping the header
For now, we are skipping the header
For now, we are skipping the header
For now, we are skipping the header
For now, we are skipping the header
For now, we are skipping the header


/Users/chitrakgupta/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [3]:
egg.shape

(8373, 11)

In [4]:
def getResults(X_train, Y_train, X_test, Y_test, preds_train, preds_test): 
    X_train_2 = X_train.copy()
    X_test_2 = X_test.copy()
    
    X_train_2['Label'] = Y_train
    X_train_2['Pred'] = preds_train
    X_train_2['Correct'] = X_train_2.Label == X_train_2.Pred
    X_train_2['Correct_2'] = X_train_2['Correct']
    X_train_2.loc[(X_train_2.Correct) & (X_train_2.Label==1.0), 'Correct_2'] = 'True binder'
    X_train_2.loc[(X_train_2.Correct) & (X_train_2.Label==0.0), 'Correct_2'] = 'True non-binder'
    X_train_2.loc[~(X_train_2.Correct) & (X_train_2.Label==1.0), 'Correct_2'] = 'False non-binder'
    X_train_2.loc[~(X_train_2.Correct) & (X_train_2.Label==0.0), 'Correct_2'] = 'False binder'

    X_test_2['Label'] = Y_test
    X_test_2['Pred'] = preds_test
    X_test_2['Correct'] = X_test_2.Label == X_test_2.Pred
    X_test_2['Correct_2'] = X_test_2['Correct']
    X_test_2.loc[(X_test_2.Correct) & (X_test_2.Label==1.0), 'Correct_2'] = 'True binder'
    X_test_2.loc[(X_test_2.Correct) & (X_test_2.Label==0.0), 'Correct_2'] = 'True non-binder'
    X_test_2.loc[~(X_test_2.Correct) & (X_test_2.Label==1.0), 'Correct_2'] = 'False non-binder'
    X_test_2.loc[~(X_test_2.Correct) & (X_test_2.Label==0.0), 'Correct_2'] = 'False binder'
    
    X_all = pd.concat([X_train_2, X_test_2])
    return X_all

In [5]:
X_train, Y_train, X_test, Y_test, bindData = g.getTrainTest(egg, cutoff = 0.15, test_size=0.33, dropDP=True)
clf_egg = svm.SVC(verbose=False)
clf_egg.fit(X_train, Y_train)
preds_train = clf_egg.predict(X_train)
preds_test = clf_egg.predict(X_test)

egg_self = getResults(X_train, Y_train, X_test, Y_test, preds_train, preds_test)

/Users/chitrakgupta/Desktop/ASU_projects/GlycanModification/Data_Mar2020/find_examples/Results_Mar2021/glycan/_prepare.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [6]:
g2 = glycan()

numberedExcel2 = g2.getData('../../MDCK_sheet2.csv', totalTabs=1)
numberedExcel3 = g2.getData('../../MDCK_sheet3.csv', totalTabs=1)
numberedExcel4 = g2.getData('../../MDCK_sheet4.csv', totalTabs=2)

numberedExcel_temp = pd.concat([numberedExcel2, numberedExcel3, numberedExcel4], axis=0)
mdck = numberedExcel_temp

mdck.reset_index(drop=True, inplace=True)
mdck['MVF'] = mdck.MVF/mdck.GlycDen

For now, we are skipping the header
For now, we are skipping the header
For now, we are skipping the header
For now, we are skipping the header


/Users/chitrakgupta/Desktop/ASU_projects/GlycanModification/Data_Mar2020/find_examples/Results_Mar2021/glycan/_read.py:162: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [7]:
X_train_mdck, Y_train_mdck, X_test_mdck, Y_test_mdck, bindData_mdck = g2.getTrainTest(mdck, cutoff = 0.05, dropDP=True, test_size=0.33)
clf_mdck = svm.SVC(verbose=False)
clf_mdck.fit(X_train_mdck, Y_train_mdck)
preds_train_mdck = clf_mdck.predict(X_train_mdck)
preds_test_mdck = clf_mdck.predict(X_test_mdck)

mdck_self = getResults(X_train_mdck, Y_train_mdck, X_test_mdck, Y_test_mdck, preds_train_mdck, preds_test_mdck)

/Users/chitrakgupta/Desktop/ASU_projects/GlycanModification/Data_Mar2020/find_examples/Results_Mar2021/glycan/_prepare.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [8]:
preds_train_cross_egg = clf_mdck.predict(X_train)
preds_test_cross_egg = clf_mdck.predict(X_test)
egg_cross = getResults(X_train, Y_train, X_test, Y_test, preds_train_cross_egg, preds_test_cross_egg)

In [9]:
see1 = egg_cross[['Pred']]
see1.rename({'Pred': 'Pred_cross'}, axis=1, inplace=True)
egg_final = egg_self.join(see1)

/Users/chitrakgupta/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [10]:
preds_train_cross_mdck = clf_egg.predict(X_train_mdck)
preds_test_cross_mdck = clf_egg.predict(X_test_mdck)
mdck_cross = getResults(X_train_mdck, Y_train_mdck, X_test_mdck, Y_test_mdck, preds_train_cross_mdck, preds_test_cross_mdck)

In [11]:
see2 = mdck_cross[['Pred']]
see2.rename({'Pred': 'Pred_cross'}, axis=1, inplace=True)
mdck_final = mdck_self.join(see2)

/Users/chitrakgupta/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [12]:
egg_final

,PrintPol,2-3 SiaLac,2-6 SiaLac,Valency,GlycSpace,GlycDen,Label,Pred,Correct,Correct_2,Pred_cross
6128,0.0,0.0,1.0,0.174757,0.625000,0.005718,1.0,1.0,True,True binder,1.0
4198,0.0,0.0,1.0,0.000000,0.000000,0.004823,1.0,1.0,True,True binder,1.0
7728,0.0,0.0,1.0,0.000000,0.000000,0.018995,1.0,1.0,True,True binder,1.0
1828,0.0,0.0,1.0,0.592233,0.441860,0.005556,1.0,1.0,True,True binder,1.0
750,0.0,0.0,1.0,0.592233,0.441860,0.006006,1.0,1.0,True,True binder,1.0
...,...,...,...,...,...,...,...,...,...,...,...
6067,1.0,1.0,0.0,1.000000,0.508721,0.056311,0.0,0.0,True,True non-binder,1.0
7021,1.0,1.0,0.0,1.000000,0.508721,0.082877,0.0,0.0,True,True non-binder,1.0
6071,1.0,1.0,0.0,1.000000,0.508721,0.068382,0.0,0.0,True,True non-binder,1.0
6069,1.0,1.0,0.0,1.000000,0.508721,0.059120,0.0,0.0,True,True non-binder,1.0


In [13]:
egg = egg[egg.GlycType != 'Lac']
egg['Status'] = 'Others'

egg.loc[((egg_final.Correct_2 == 'True binder') & (egg_final.Pred_cross == 1.0)), 'Status'] = 'Conserved'
egg.loc[((egg_final.Correct_2 == 'True non-binder') & (egg_final.Pred_cross == 1.0)), 'Status'] = 'Lost in egg'
egg.loc[((egg_final.Correct_2 == 'True binder') & (egg_final.Pred_cross == 0.0)), 'Status'] = 'Gained in egg'

In [14]:
eggcopy = egg.copy()
eggcopy[['Label', 'Pred', 'Pred_cross']] = egg_final[['Label', 'Pred', 'Pred_cross']]

In [15]:
mdck = mdck[mdck.GlycType != 'Lac']
mdck['Status'] = 'Others'

mdck.loc[((mdck_final.Correct_2 == 'True binder') & (mdck_final.Pred_cross == 1.0), 'Status')] = 'Conserved'
mdck.loc[((mdck_final.Correct_2 == 'True non-binder') & (mdck_final.Pred_cross == 1.0), 'Status')] = 'Lost in MDCK'
mdck.loc[((mdck_final.Correct_2 == 'True binder') & (mdck_final.Pred_cross == 0.0), 'Status')] = 'Gained in MDCK'

/Users/chitrakgupta/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/chitrakgupta/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [16]:
mdckcopy = mdck.copy()
mdckcopy[['Label', 'Pred', 'Pred_cross']] = mdck_final[['Label', 'Pred', 'Pred_cross']]

In [17]:
egg_Mar21 = eggcopy.drop(['SubArr', 'ExpNum', 'FileName'], axis=1)
egg_Mar21['Experiment'] = 'Non-binder'
egg_Mar21['Self-model'] = 'Non-binder'
egg_Mar21['Cross-model'] = 'Non-binder'
egg_Mar21.loc[(egg_Mar21.Label == 1), 'Experiment'] = 'Binder'
egg_Mar21.loc[(egg_Mar21.Pred == 1), 'Self-model'] = 'Binder'
egg_Mar21.loc[(egg_Mar21.Pred_cross == 1), 'Cross-model'] = 'Binder'
egg_Mar21_final = egg_Mar21.drop(['Label', 'Pred', 'Pred_cross'], axis=1)

In [18]:
mdck_Mar21 = mdckcopy.drop(['SubArr', 'ExpNum', 'FileName'], axis=1)
mdck_Mar21['Experiment'] = 'Non-binder'
mdck_Mar21['Self-model'] = 'Non-binder'
mdck_Mar21['Cross-model'] = 'Non-binder'
mdck_Mar21.loc[(mdck_Mar21.Label == 1), 'Experiment'] = 'Binder'
mdck_Mar21.loc[(mdck_Mar21.Pred == 1), 'Self-model'] = 'Binder'
mdck_Mar21.loc[(mdck_Mar21.Pred_cross == 1), 'Cross-model'] = 'Binder'
mdck_Mar21_final = mdck_Mar21.drop(['Label', 'Pred', 'Pred_cross'], axis=1)

In [19]:
egg_Mar21_final.to_csv("egg_allcolumns_Mar21.csv", index=False)
mdck_Mar21_final.to_csv("mdck_allcolumns_Mar21.csv", index=False)

In [23]:
egg_cnb = egg_Mar21_final.loc[((egg_Mar21_final['Experiment'] == 'Non-binder') & (egg_Mar21_final['Self-model'] == 'Non-binder') & (egg_Mar21_final['Cross-model'] == 'Non-binder'))]

In [24]:
mdck_cnb = mdck_Mar21_final.loc[((mdck_Mar21_final['Experiment'] == 'Non-binder') & (mdck_Mar21_final['Self-model'] == 'Non-binder') & (mdck_Mar21_final['Cross-model'] == 'Non-binder'))]

In [25]:
egg_cnb.to_csv("egg_conserved_nonbinders.csv", index=False)
mdck_cnb.to_csv("mdck_conserved_nonbinders.csv", index=False)

In [26]:
egg.to_csv("egg_status_Feb1.csv", index=False)
mdck.to_csv("mdck_status_Feb1.csv", index=False)

In [27]:
egg_reset = egg.reset_index(drop=True)
lastInd = egg_reset.index[-1] + 1

In [28]:
mdck_reset = mdck.reset_index(drop=True)
mdck_reset.index = mdck_reset.index + lastInd

In [29]:
allDat = pd.concat([egg_reset, mdck_reset])
allDat.to_csv("All_Feb1.csv", index=False)

In [30]:
eggcopy.to_csv("Egg_Feb1.csv", index=False)

In [31]:
mdckcopy.to_csv("MDCK_Feb1.csv", index=False)

In [32]:
mdckcopy[mdckcopy.Status == 'Gained in MDCK']

,PrintPol,GlycType,DP,Valency,GlycSpace,PolSpace,GlycDen,MVF,SubArr,ExpNum,FileName,Status,Label,Pred,Pred_cross
30,1.0,2-6 SiaLac,300.0,141.0,0.47,0.354,49.861,4.713102,0,6,../../MDCK_sheet2.csv,Gained in MDCK,1.0,1.0,0.0
31,1.0,2-6 SiaLac,300.0,141.0,0.47,0.328,46.265,5.446882,0,6,../../MDCK_sheet2.csv,Gained in MDCK,1.0,1.0,0.0
32,1.0,2-6 SiaLac,300.0,141.0,0.47,0.325,45.772,5.177838,0,6,../../MDCK_sheet2.csv,Gained in MDCK,1.0,1.0,0.0
33,1.0,2-6 SiaLac,300.0,141.0,0.47,0.348,49.119,5.150756,0,6,../../MDCK_sheet2.csv,Gained in MDCK,1.0,1.0,0.0
34,1.0,2-6 SiaLac,300.0,141.0,0.47,0.404,56.969,4.353245,0,6,../../MDCK_sheet2.csv,Gained in MDCK,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1214,10.0,2-3 SiaLac,300.0,147.0,2.04,0.058,8.592,400.837989,3,99,../../MDCK_sheet4.csv,Gained in MDCK,1.0,1.0,0.0
1215,10.0,2-3 SiaLac,300.0,147.0,2.04,0.035,5.159,738.321380,3,99,../../MDCK_sheet4.csv,Gained in MDCK,1.0,1.0,0.0
1216,10.0,2-3 SiaLac,300.0,147.0,2.04,0.060,8.850,395.028249,3,99,../../MDCK_sheet4.csv,Gained in MDCK,1.0,1.0,0.0
1217,10.0,2-3 SiaLac,300.0,147.0,2.04,0.065,9.557,395.103066,3,99,../../MDCK_sheet4.csv,Gained in MDCK,1.0,1.0,0.0


In [33]:
first_elim = mdck.loc[(mdck_final.Label == 1) & (mdck_final.Pred == 0)]

In [34]:
second_elim = mdck.loc[(mdck_final.Label == 0) & (mdck_final.Pred == 1)]

In [35]:
third_elim = mdck.loc[(mdck_final.Label == 0) & (mdck_final.Pred == 0) & (mdck_final.Pred_cross == 0)]

In [36]:
first_elim.to_csv("FirstElim.csv", index=False)

In [37]:
second_elim.to_csv("SecondElim.csv", index=False)

In [38]:
third_elim.to_csv("ThirdElim.csv", index=False)